This is my  first attemp in data analysis. I have just started a week back and have gone through the basic tutorials in python for data analysis and visualization. Any suggestions on improvement are welcome :)  
Let me first load the train and test files  and understand the data. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
train_data = pd.read_csv("../input/train.csv")
test_data = pd.read_csv("../input/test.csv")
print(train_data.shape)
print(test_data.shape)
print(train_data.columns)
ob_col= (train_data.select_dtypes(include=['object'])).columns
print(ob_col)
print(train_data.describe())

So there are  4209 rows od data with 378 columns in the train data and 377 columns in the test data. The extra column in the train data being the prediction variable 'y'.
There are 8 columns which are of type object. All the remaining columns are float type.

When we do .describe() we can get the fair view of the data - How the column values are spread.  Wait! The column X11 is all 0s. This will not help much in prediction. Lets remove the columns which have just 1 value for every row.

In [ ]:
for col in train_data.columns:
    if len(train_data[col].unique()) == 1:
        del train_data[col]
print(train_data.columns)

Now we are down to 366 columns in train_data. Make sure these columns are removed in the test_data as well. or else it might cause some errors when we run the prediction algorithms.

In [ ]:
y = train_data['y']
X = train_data.drop(['y'], axis=1)

test_data_refined = test_data[X.columns]
print(X.columns)
print(test_data_refined.columns)

Now lets understand the other 8 columns with dtype object. And lets see how they are spread. 

In [ ]:
import matplotlib.pyplot as plt
fig, axarr = plt.subplots(8, 1, figsize=(10, 30))
i=0
for c in ob_col:
    X[c].value_counts().sort_index().plot.bar(
    ax=axarr[i], fontsize=12, color='mediumvioletred')
    axarr[i].set_title(c, fontsize=12)
    i=i+1

With the above graphs we can see how the predictor variables are spread. For example X4 has many d's.  Now lets convert these columns with dtypes objects to integres which can be inserted to the Randomforest algorithm.  

In [ ]:
train_data_onehot = pd.get_dummies(X)
test_data_onehot = pd.get_dummies(test_data_refined)
train_predictors,test_predictors = train_data_onehot.align(test_data_onehot,join='inner',axis=1)

print(train_predictors.shape)
print(test_predictors.shape)

Let us use the RandomForest to predict

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(train_predictors,y)
predictions = model.predict(test_predictors)

Let us write the predictions in outputfile before submitting

In [ ]:
ans_df = pd.DataFrame({'ID': test_data.ID, 'y': predictions})
print(ans_df.describe())
ans_df.to_csv('answer.csv', index=False)

Next we will see how can we fine tune the algorithm for better accuracy